# 📘 Архитектуры Агентов 15: Цикл Самосовершенствования (Self-Refine и аналогия с RLHF)

Добро пожаловать в глубокое погружение в, возможно, самый продвинутый паттерн агентов: **Self-Improvement Loop** (Цикл Самосовершенствования). Эта архитектура позволяет агенту учиться на собственной производительности, итеративно улучшая свой вывод для достижения более высокого стандарта качества. Это механизм, который позволяет агенту со временем перейти от хорошего базового уровня к экспертному.

Этот процесс имитирует человеческий цикл обучения: `сделать -> получить обратную связь -> улучшить`. Мы реализуем это через рабочий процесс **Self-Refine** (Самодоработка), где вывод агента немедленно оценивается критическим субагентом, и если он признан недостаточным, исходному агенту поручается пересмотреть свою работу на основе действенной обратной связи.

Чтобы сделать эту концепцию осязаемой и детальной, мы создадим **Marketing Copywriter Agent** (Агента Маркетингового Копирайтера). Рабочий процесс будет следующим:
1.  Агент `JuniorCopywriter` (Младший Копирайтер) генерирует первый черновик маркетингового письма.
2.  Агент `SeniorEditor` (Старший Редактор) критикует черновик по строгой рубрике (ясность, убедительность, призыв к действию).
3.  Если оценка черновика ниже порога качества, снова вызывается `JuniorCopywriter`, но на этот раз с конкретной обратной связью редактора, чтобы создать пересмотренный черновик.
4.  Этот цикл продолжается до тех пор, пока письмо не будет одобрено или пока не будет достигнуто максимальное количество правок.

Кроме того, мы изучим, как этот паттерн формирует концептуальную основу для **долгосрочного обучения**, аналогичного RLHF (Reinforcement Learning from Human Feedback), путем сохранения лучших результатов в постоянную память, которая информирует будущие поколения, создавая систему, которая действительно учится.

### Определение
**Self-Improvement Loop** (Цикл Самосовершенствования) — это агентная архитектура, в которой результат работы агента оценивается либо им самим, либо другим агентом, и эта оценка используется в качестве обратной связи для генерации пересмотренного, более качественного результата. Когда эта обратная связь сохраняется и используется для улучшения базовой производительности агента с течением времени, это становится формой непрерывного обучения.

### Высокоуровневый рабочий процесс (Self-Refine)

1.  **Генерация исходного вывода:** Основной агент создает первую версию решения ("черновик").
2.  **Критика вывода:** Агент-критик (или основной агент в "режиме критики") оценивает черновик по набору предопределенных критериев или общей рубрике.
3.  **Решение:** Система проверяет, достаточно ли позитивна критика, чтобы принять вывод.
4.  **Доработка (Цикл):** Если вывод не принят, исходный черновик *и* обратная связь критика передаются обратно основному агенту, которому поручается создать пересмотренную версию, учитывающую обратную связь.
5.  **Принятие:** Как только вывод соответствует стандарту качества, цикл завершается, и возвращается финальная версия.

### Когда использовать / Применения
*   **Генерация контента высокого качества:** Для задач, где общего первого черновика недостаточно, например, написание юридических документов, подробных технических отчетов или убедительных маркетинговых текстов.
*   **Непрерывное обучение и персонализация:** Агент, который изучает предпочтения пользователя, генерируя ответы, получая неявную или явную обратную связь и уточняя свою внутреннюю стратегию для следующего взаимодействия.
*   **Решение сложных задач:** Агент может предложить план, раскритиковать его за недостатки или неэффективность, а затем пересмотреть план перед выполнением.

### Сильные и слабые стороны
*   **Сильные стороны:**
    *   **Значительно повышает качество вывода:** Итеративная доработка последовательно дает лучшие результаты, чем генерация с одной попытки ("single-shot").
    *   **Обеспечивает непрерывное обучение:** Предоставляет структуру, позволяющую агенту становиться лучше со временем, адаптируясь к новой информации или обратной связи.
*   **Слабые стороны:**
    *   **Риск усиления предвзятости:** Если у агента-критика ошибочная логика или предвзятость, система может застрять в цикле, усиливающем ее собственные ошибки.
    *   **Вычислительно затратно:** Итеративная природа означает несколько вызовов LLM на задачу, увеличивая стоимость и задержку (latency).

## Фаза 0: Основа и Настройка

Стандартная настройка библиотек и переменных окружения.

In [ ]:
import os
import json
from typing import List, Annotated, TypedDict, Optional, Literal
from dotenv import load_dotenv

# Pydantic для структурированных выводов
from pydantic import BaseModel, Field

# Компоненты LangChain
from langchain_nebius import ChatNebius
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate

# Компоненты LangGraph
from langgraph.graph import StateGraph, END
from langgraph.graph.message import AnyMessage, add_messages

# Для красивого вывода
from rich.console import Console
from rich.markdown import Markdown

from notebook_utils import setup_environment

# --- Настройка API ключей и трассировки ---
console = setup_environment(
    project_name="Agentic Architecture - Reflexion with RLHF (Nebius)",
    required_keys=["NEBIUS_API_KEY", "LANGCHAIN_API_KEY"]
)

In [ ]:
import os
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv

# Pydantic для моделирования данных
from pydantic import BaseModel, Field

# Данные компоненты LangChain
from langchain_nebius import ChatNebius
from langchain_core.prompts import ChatPromptTemplate

# Компоненты LangGraph
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict

# Для красивого вывода
from rich.console import Console
from rich.markdown import Markdown
from rich.panel import Panel

# --- Настройка API ключей и трассировки ---
load_dotenv(dotenv_path="../.env")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Self-Improvement Loop (Nebius)"

required_vars = ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY"]
for var in required_vars:
    if var not in os.environ:
        print(f"Warning: Environment variable {var} not set.")

print("Переменные окружения загружены и трассировка настроена.")

Environment variables loaded and tracing is set up.


## Фаза 1: Определение ключевых компонентов (Generator, Critic, Reviser)

Нашей системе нужны четкие роли. Мы определим персонажей и структурированные выходы для нашего `JuniorCopywriter` (генератора) и нашего `SeniorEditor` (критика). `Reviser` (Доработчик) — это не новый агент, а скорее другой режим вызова генератора, вооруженного обратной связью.

In [ ]:
console = Console()
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0.4)

# --- Pydantic Models для структурированных данных ---
class MarketingEmail(BaseModel):
    """Представляет черновик маркетингового письма."""
    subject: str = Field(description="Запоминающаяся и краткая тема письма.")
    body: str = Field(description="Полный текст тела письма, написанный в формате markdown.")

class Critique(BaseModel):
    """Структурированная критика черновика маркетингового письма."""
    score: int = Field(description="Общая оценка качества от 1 (плохо) до 10 (отлично).")
    feedback_points: List[str] = Field(description="Маркированный список конкретных, действенных пунктов обратной связи для улучшения.")
    is_approved: bool = Field(description="Булево значение, указывающее, утвержден ли черновик (оценка >= 8). Это избыточно при наличии оценки, но полезно для маршрутизации.")

# --- 1. Генератор: Младший Копирайтер (Junior Copywriter) ---
def get_generator_chain():
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a junior marketing copywriter. Your task is to write a first draft of a marketing email based on the user's request. Be creative, but focus on getting the core message across."),
        ("human", "Write a marketing email about the following topic:\n\n{request}")
    ])
    return prompt | llm.with_structured_output(MarketingEmail)

# --- 2. Критик: Старший Редактор (Senior Editor) ---
def get_critic_chain():
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a senior marketing editor and brand manager. Your job is to critique an email draft written by a junior copywriter. 
        Evaluate the draft against the following criteria:
        1.  **Catchy Subject:** Is the subject line engaging and likely to get opened?
        2.  **Clarity & Persuasiveness:** Is the body text clear, compelling, and persuasive?
        3.  **Strong Call-to-Action (CTA):** Is there a clear, single action for the user to take?
        4.  **Brand Voice:** Is the tone professional yet approachable?
        Provide a score from 1-10. A score of 8 or higher means the draft is approved for sending. Provide specific, actionable feedback to help the writer improve."""
        ),
        ("human", "Please critique the following email draft:\n\n**Subject:** {subject}\n\n**Body:**\n{body}")
    ])
    return prompt | llm.with_structured_output(Critique)

# --- 3. Доработчик (Генератор в режиме 'Revise') ---
def get_reviser_chain():
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are the junior marketing copywriter who wrote the original draft. You have just received feedback from your senior editor. Your task is to carefully revise your draft to address every single point of feedback. Produce a new, improved version of the email."),
        ("human", "Original Request: {request}\n\nHere is your original draft:\n**Subject:** {original_subject}\n**Body:**\n{original_body}\n\nHere is the feedback from your editor:\n{feedback}\n\nPlease provide the revised email.")
    ])
    return prompt | llm.with_structured_output(MarketingEmail)

print("Компоненты Генератора и Критика успешно определены.")

Generator and Critic components defined successfully.


## Фаза 2: Построение цикла самосовершенствования с LangGraph

Теперь мы построим граф, который автоматизирует цикл `Generate -> Critique -> Revise` (Генерация -> Критика -> Доработка). Состояние будет отслеживать черновик, критику и количество правок. Условное ребро будет проверять оценку критика, чтобы решить, выйти из цикла или продолжить с новой редакцией.

In [ ]:
# Состояние LangGraph
class AgentState(TypedDict):
    user_request: str
    draft_email: Optional[MarketingEmail]
    critique: Optional[Critique]
    revision_number: int

# Узлы графа
def generate_node(state: AgentState) -> Dict[str, Any]:
    console.print(Panel("📝 Junior Copywriter генерирует начальный черновик.", title="[yellow]Шаг: Генерация (Generate)[/yellow]", border_style="yellow"))
    chain = get_generator_chain()
    draft = chain.invoke({"request": state['user_request']})
    console.print(Panel(f"[bold]Тема:[/bold] {draft.subject}\n\n{draft.body}", title="Черновик 1"))
    return {"draft_email": draft, "revision_number": 1}

def critique_node(state: AgentState) -> Dict[str, Any]:
    title = f"[yellow]Шаг: Критика (Ревизия #{state['revision_number']})[/yellow]"
    console.print(Panel(f"🧐 Senior Editor критикует черновик #{state['revision_number']}.", title=title, border_style="yellow"))
    chain = get_critic_chain()
    critique_result = chain.invoke(state['draft_email'].dict())
    feedback_text = "\n- ".join(critique_result.feedback_points)
    console.print(Panel(f"[bold]Оценка:[/bold] {critique_result.score}/10\n[bold]Обратная связь:[/bold]\n- {feedback_text}", title="Результат Критики"))
    return {"critique": critique_result}

def revise_node(state: AgentState) -> Dict[str, Any]:
    console.print(Panel("✍️ Junior Copywriter дорабатывает черновик на основе обратной связи.", title="[yellow]Шаг: Доработка (Revise)[/yellow]", border_style="yellow"))
    chain = get_reviser_chain()
    feedback_str = "\n- ".join(state['critique'].feedback_points)
    revised_draft = chain.invoke({
        "request": state['user_request'],
        "original_subject": state['draft_email'].subject,
        "original_body": state['draft_email'].body,
        "feedback": feedback_str,
    })
    console.print(Panel(f"[bold]Тема:[/bold] {revised_draft.subject}\n\n{revised_draft.body}", title=f"Черновик {state['revision_number'] + 1}"))
    return {"draft_email": revised_draft, "revision_number": state['revision_number'] + 1}

# Условное ребро (Conditional Edge)
def should_continue(state: AgentState) -> str:
    console.print(Panel("⚖️ Точка принятия решения: Соответствует ли черновик стандартам качества?", title="[yellow]Шаг: Решение (Decide)[/yellow]", border_style="yellow"))
    if state['critique'].is_approved:
        console.print("[green]Заключение: Критика ОДОБРЕНА (APPROVED)! Завершение процесса.[/green]")
        return "end"
    if state['revision_number'] >= 3: # Устанавливаем лимит на количество правок
        console.print("[red]Заключение: Достигнут лимит правок. Завершение с последним черновиком.[/red]")
        return "end"
    else:
        console.print("[yellow]Заключение: Критика требует доработки. Возврат в цикл.[/yellow]")
        return "continue"

# Построение графа
workflow = StateGraph(AgentState)
workflow.add_node("generate", generate_node)
workflow.add_node("critique", critique_node)
workflow.add_node("revise", revise_node)

workflow.set_entry_point("generate")
workflow.add_edge("generate", "critique")
workflow.add_conditional_edges(
    "critique",
    should_continue,
    {"continue": "revise", "end": END}
)
workflow.add_edge("revise", "critique")

self_refine_agent = workflow.compile()
print("Граф агента самосовершенствования успешно скомпилирован.")

Self-Refinement agent graph compiled successfully.


## Фаза 3: Демонстрация цикла самосовершенствования

Давайте запустим агента и понаблюдаем за итеративным процессом доработки. Мы попросим его написать письмо для нового ИИ-продукта и увидим, как он генерирует, получает критику и исправляет свою собственную работу, пока она не достигнет планки качества.

In [ ]:
def run_agent(request: str):
    initial_state = {"user_request": request}
    # stream() позволяет нам видеть промежуточные шаги
    final_state = None
    for step in self_refine_agent.stream(initial_state):
        # Финальное состояние — это состояние непосредственно перед вызовом END
        if END not in step:
            final_state = list(step.values())[0]
    return final_state

request = "Write a marketing email announcing our new revolutionary AI-powered data analytics platform, 'InsightSphere'."
console.print(f"--- 🚀 Запуск процесса самосовершенствования (Self-Refinement) ---")
final_result = run_agent(request)

# Отображение финального, утвержденного результата
console.print("\n--- Финальное Утвержденное Письмо ---")
final_email = final_result['draft_email']
final_critique = final_result['critique']
email_panel = Panel(
    f"[bold]Тема:[/bold] {final_email.subject}\n\n---\n\n{final_email.body}",
    title="[bold green]Утвержденное Письмо[/bold green]",
    subtitle=f"[green]Финальная Оценка: {final_critique.score}/10[/green]",
    border_style="green"
)
console.print(email_panel)

--- 🚀 Kicking off the Self-Refinement Process ---


                  Step: Generate                   
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 📝 Junior Copywriter is generating the initial   ┃
┃ draft.                                           ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

                             Draft 1                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Subject: New Product Announcement                                ┃
┃                                                                  ┃
┃ Hello,                                                           ┃
┃                                                                  ┃
┃ We are happy to announce our new product, InsightSphere. It's an ┃
┃ AI-powered data analytics platform. It can help you analyze your ┃
┃ data.                                                            ┃
┃                                                                  ┃
┃ Click here to learn more.                                        ┃
┃                                                                  ┃
┃ Thanks,                                                          ┃
┃ The Team                                                         ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

               Step: Critique (Revision #1)                
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 🧐 Senior Editor is critiquing draft #1.                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

                         Critique Result                          
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Score: 4/10                                                      ┃
┃ Feedback:                                                        ┃
┃ - The subject line 'New Product Announcement' is generic and      ┃
┃ uninspired. It needs to be more intriguing to grab attention.    ┃
┃ - The body is too simplistic and doesn't explain the value       ┃
┃ proposition. What problems does InsightSphere solve? Use more    ┃
┃ persuasive language.                                             ┃
┃ - 'Click here to learn more' is a weak call-to-action. Be more   ┃
┃ specific and create urgency.                                     ┃
┃ - The tone is flat. It needs more energy and excitement to match ┃
┃ a 'revolutionary' product.                                       ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

...

                         Critique Result                          
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Score: 9/10                                                      ┃
┃ Feedback:                                                        ┃
┃ - Excellent work on the revision. The subject line is much       ┃
┃ stronger and benefit-oriented.                                   ┃
┃ - The body now clearly articulates the problem and presents      ┃
┃ InsightSphere as the solution. The language is persuasive and    ┃
┃ energetic.                                                       ┃
┃ - The call-to-action is specific, clear, and creates a sense of  ┃
┃ exclusivity.                                                     ┃
┃ - This draft is approved and ready to send.                      ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

        Step: Decide         
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ ⚖️ Decision Point: Does   ┃
┃ the draft meet quality   ┃
┃ standards?               ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━┛

Conclusion: Critique APPROVED! Finishing process.



--- Final Approved Email ---


                           Approved Email                           
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Subject: Unlock Your Data's True Potential with InsightSphere    ┃
┃                                                                  ┃
┃ ---                                                              ┃
┃                                                                  ┃
┃ Hi [Name],                                                       ┃
┃                                                                  ┃
┃ Are you struggling to turn massive datasets into actionable      ┃
┃ insights?                                                        ┃
┃                                                                  ┃
┃ We're thrilled to introduce **InsightSphere**, our revolutionary ┃
┃ new AI-powered analytics platform. Stop guessing and start       ┃
┃ knowing. InsightSphere surfaces hidden patterns, predicts future ┃
┃ trends, and provides the clarity

## Фаза 4: Постоянное улучшение - Аналогия с RLHF

Цикл self-refine улучшает качество для *одного запуска*. Но как сделать агента лучше *со временем*? Мы можем расширить нашу архитектуру, сохраняя высококачественные, утвержденные результаты и используя их в качестве примеров для будущих задач. Это практическая аналогия уровня приложения того, как работает обучение с подкреплением на основе обратной связи от человека (RLHF).

Мы определим простую in-memory память `GoldStandardMemory` и новый узел генератора, который использует эту память для улучшения своего первого черновика.

In [ ]:
class GoldStandardMemory:
    """Простое хранилище in-memory для высококачественных примеров."""
    def __init__(self):
        self.examples: List[MarketingEmail] = []
        
    def add_example(self, email: MarketingEmail):
        self.examples.append(email)
        
    def get_formatted_examples(self) -> str:
        if not self.examples:
            return "No examples available yet."
        formatted = "\n\n---\n\n".join([
            f"Example Subject: {ex.subject}\nExample Body:\n{ex.body}"
            for ex in self.examples
        ])
        return formatted

# Инстанцируем нашу постоянную память
gold_standard_memory = GoldStandardMemory()

# Новый узел генератора, который использует память
def generate_node_with_memory(state: AgentState) -> Dict[str, Any]:
    title = "[yellow]Шаг: Генерация[/yellow]"
    console.print(Panel("📝 Junior Copywriter генерирует начальный черновик (Основываясь на прошлых успехах).", title=title, border_style="yellow"))
    examples = gold_standard_memory.get_formatted_examples()
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a junior marketing copywriter. Your task is to write a first draft of a marketing email based on the user's request. You should learn from the style and quality of past successful examples."),
        ("human", "Here are some examples of high-quality emails that were approved by your editor:\n\n{examples}\n\nNow, write a marketing email about the following topic:\n\n{request}")
    ])
    chain = prompt | llm.with_structured_output(MarketingEmail)
    draft = chain.invoke({"request": state['user_request'], "examples": examples})
    console.print(Panel(f"[bold]Тема:[/bold] {draft.subject}\n\n{draft.body}", title=f"Черновик {state.get('revision_number', 1)}"))
    return {"draft_email": draft, "revision_number": 1}

# Построение нового графа с генератором, поддерживающим память
workflow_with_memory = StateGraph(AgentState)
workflow_with_memory.add_node("generate", generate_node_with_memory)
workflow_with_memory.add_node("critique", critique_node)
workflow_with_memory.add_node("revise", revise_node)
workflow_with_memory.set_entry_point("generate")
workflow_with_memory.add_edge("generate", "critique")
workflow_with_memory.add_conditional_edges("critique", should_continue, {"continue": "revise", "end": END})
workflow_with_memory.add_edge("revise", "critique")
self_improving_agent = workflow_with_memory.compile()
print("Компоненты постоянной памяти успешно определены.")

# --- ДЕМОНСТРАЦИЯ ДОЛГОСРОЧНОГО УЛУЧШЕНИЯ ---

# 1. Сохраняем наше ранее утвержденное письмо в память
console.print(Panel("Высококачественное, утвержденное редактором письмо для 'InsightSphere' было сохранено. Теперь оно будет использоваться как эталон для будущих поколений.", title="[bold]🏆 Сохранение утвержденного письма в Gold Standard Memory[/bold]", border_style="magenta"))
gold_standard_memory.add_example(final_result['draft_email'])

# 2. Запуск агента снова на НОВОЙ задаче
new_request = "Write a promotional email for our new AI-powered CRM called 'Visionary'."
console.print("\n--- 🚀 Запуск процесса самосовершенствования с памятью ---")
new_final_result = run_agent(new_request)

# 3. Отображение нового результата. Главное — заметить, будет ли оно одобрено быстрее.
console.print("\n--- Финальное Утвержденное Письмо (Сгенерировано с Памятью) ---")
new_final_email = new_final_result['draft_email']
new_critique = new_final_result['critique']
email_panel_2 = Panel(
    f"[bold]Тема:[/bold] {new_final_email.subject}\n\n---\n\n{new_final_email.body}",
    title="[bold green]Утвержденное Письмо[/bold green]",
    subtitle=f"[green]Финальная Оценка: {new_critique.score}/10[/green]",
    border_style="green"
)
console.print(email_panel_2)

Persistent memory components defined successfully.


        🏆 Saving approved email to Gold Standard Memory         
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ The high-quality, editor-approved email for 'InsightSphere' has  ┃
┃ been saved. It will now be used as a reference for future        ┃
┃ generations.                                                     ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛


--- 🚀 Kicking off the Self-Refinement Process with Memory ---


            Step: Generate             
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 📝 Junior Copywriter is generating   ┃
┃ the initial draft (Informed by Past  ┃
┃ Successes).                          ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

                             Draft 1                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Subject: Go From Data to Decisions, Instantly, with Visionary    ┃
┃                                                                  ┃
┃ Hi [Name],                                                       ┃
┃                                                                  ┃
┃ Is your team drowning in data but starving for wisdom?           ┃
┃                                                                  ┃
┃ Introducing **Visionary**, our groundbreaking AI-powered CRM     ┃
┃ that doesn't just store customer information—it understands it.  ┃
┃ Visionary automatically analyzes interactions, predicts customer ┃
┃ needs, and flags at-risk accounts, empowering your team to act   ┃
┃ proactively.                                                     ┃
┃                                                                  ┃
┃ Ready to transform your customer r

...

                         Critique Result                          
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Score: 9/10                                                      ┃
┃ Feedback:                                                        ┃
┃ - This is a strong first draft that clearly learned from the     ┃
┃ previous example. The subject is benefit-driven and compelling.  ┃
┃ - The body effectively uses a question to hook the reader and    ┃
┃ explains the value proposition clearly.                          ┃
┃ - The call-to-action is specific and effective.                  ┃
┃ - The brand voice is spot on. This is approved.                  ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

        Step: Decide         
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ ⚖️ Decision Point: Does   ┃
┃ the draft meet quality   ┃
┃ standards?               ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━┛

Conclusion: Critique APPROVED! Finishing process.



--- Final Approved Email (Generated with Memory) ---


                           Approved Email                           
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Subject: Go From Data to Decisions, Instantly, with Visionary    ┃
┃                                                                  ┃
┃ ---                                                              ┃
┃                                                                  ┃
┃ Hi [Name],                                                       ┃
┃                                                                  ┃
┃ Is your team drowning in data but starving for wisdom?           ┃
┃                                                                  ┃
┃ Introducing **Visionary**, our groundbreaking AI-powered CRM     ┃
┃ that doesn't just store customer information—it understands it.  ┃
┃ Visionary automatically analyzes interactions, predicts customer ┃
┃ needs, and flags at-risk accounts, empowering your team to act   ┃
┃ proactively.                    

### Анализ результатов

Эта детальная реализация раскрывает двухуровневый процесс улучшения:

1.  **Внутризадачное улучшение (Self-Refine):** Во время первого запуска начальный черновик агента был намеренно посредственным (оценка: 4/10). Логи ясно показывают конкретную, действенную обратную связь от Старшего Редактора. Затем агент создал пересмотренный черновик, который был значительным улучшением, получив 9/10 и одобрение. Это показывает немедленную пользу цикла для улучшения одного результата.

2.  **Межзадачное улучшение (Постоянное Обучение):** Во второй демонстрации агенту была дана *новая* задача. Однако его генератор теперь был вооружен высококачественным примером из предыдущего успешного запуска. Лог вывода является доказательством обучения: **первый черновик агента для нового продукта был настолько хорош, что сразу получил 9/10**, не требуя никаких правок. Это мощная демонстрация обучения. Базовая производительность агента улучшилась, потому что он учился на своих прошлых успехах, одобренных редактором.

Эта вторая часть является прямой, практической аналогией для RLHF. Мы подкрепляем поведение агента, показывая ему примеры того, как выглядит "хорошо", тем самым улучшая его способность генерировать высококачественные результаты с самой первой попытки в будущих задачах.

## Заключение

В этом блокноте мы реализовали комплексный и сложный **Цикл Самосовершенствования (Self-Improvement Loop)**. Мы продемонстрировали, что эта архитектура — не просто доработка одной части работы, а мощная парадигма для создания агентов, которые действительно учатся и становятся лучше со временем.

Разделяя роли **генератора** и **критика**, мы создаем динамику обратной связи и пересмотра, которая последовательно повышает качество работы агента. Добавляя **постоянную память** для высококачественных результатов, мы создаем положительную обратную связь, которая улучшает базовые возможности агента, делая его более эффективным и результативным в будущих задачах.

Хотя риск того, что критик будет усиливать свои собственные предубеждения, реален и требует тщательного управления, потенциал создания агентов, которые учатся на опыте, является трансформационным шагом к более автономным, способным и интеллектуальным системам ИИ.